- Load Data
- Prepare Model
- Assess the Model

In [36]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.wrappers. scikit_learn import KerasClassifier
from keras.utils import np_utils

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

In [2]:
import sys
from data import *
x = completeX()
biny = binaryY()
biny = biny.astype(np.int8)

In [3]:
x.shape

(198, 43)

In [8]:
from sklearn.model_selection import train_test_split
num_val_samples = int((x.shape[0])*0.2)
x_train, x_test, y_train, y_test = train_test_split(x, 
                                                    biny, 
                                                    test_size=num_val_samples, 
                                                    random_state=42)
print("Number of training samples:", len(x_train))
print("Number of validation samples:", len(x_test))

Number of training samples: 159
Number of validation samples: 39


In [5]:
x.shape[0]

198

In [11]:
counts = np.bincount(y_test)

In [18]:
y = pd.DataFrame(biny)

In [21]:
np.unique(y)

array([0, 1], dtype=int8)

In [23]:
from collections import Counter

In [24]:
Counter(biny)

Counter({0: 151, 1: 47})

In [29]:
positive = biny[biny==1].shape[0]
negative = biny[biny==0].shape[0]
weight_for_0 = 1.0/negative
weight_for_1 = 1.0/positive

In [34]:
std = x_train.std(axis=0)
mean = x_train.mean(axis=0)

In [35]:
x_train = (x_train-mean)/std
x_test = (x_test-mean)/std

In [44]:
from tensorflow import keras

model = keras.Sequential(
    [
        keras.layers.Dense(
            10, activation="relu", input_shape=(x_train.shape[-1],)
        ),
        keras.layers.Dense(10, activation="relu"),
        keras.layers.Dropout(0.3),
        keras.layers.Dense(1, activation="sigmoid"),
    ]
)
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 10)                440       
_________________________________________________________________
dense_5 (Dense)              (None, 10)                110       
_________________________________________________________________
dropout_2 (Dropout)          (None, 10)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 11        
Total params: 561
Trainable params: 561
Non-trainable params: 0
_________________________________________________________________


In [45]:
metrics = [
    keras.metrics.FalseNegatives(name="fn"),
    keras.metrics.FalsePositives(name="fp"),
    keras.metrics.TrueNegatives(name="tn"),
    keras.metrics.TruePositives(name="tp"),
    keras.metrics.Precision(name="precision"),
    keras.metrics.Recall(name="recall"),
]

model.compile(
    optimizer=keras.optimizers.Adam(1e-2), loss="binary_crossentropy", metrics=metrics
)

callbacks = [keras.callbacks.ModelCheckpoint("fraud_model_at_epoch_{epoch}.h5")]
class_weight = {0: weight_for_0, 1: weight_for_1}

model.fit(
    x_train,
    y_train,
    batch_size=2048,
    epochs=10,
    verbose=2,
    callbacks=callbacks,
    validation_data=(x_test, y_test),
    class_weight=class_weight,
)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train on 159 samples, validate on 39 samples
Epoch 1/10
159/159 - 3s - loss: 0.0094 - fn: 39.0000 - fp: 1.0000 - tn: 118.0000 - tp: 1.0000 - precision: 0.5000 - recall: 0.0250 - val_loss: 0.0072 - val_fn: 7.0000 - val_fp: 0.0000e+00 - val_tn: 32.0000 - val_tp: 0.0000e+00 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/10
159/159 - 0s - loss: 0.0084 - fn: 37.0000 - fp: 14.0000 - tn: 105.0000 - tp: 3.0000 - precision: 0.1765 - recall: 0.0750 - val_loss: 0.0068 - val_fn: 7.0000 - val_fp: 4.0000 - val_tn: 28.0000 - val_tp: 0.0000e+00 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 3/10
159/159 - 0s - loss: 0.0079 - fn: 32.0000 - fp: 9.0000 - tn: 110.0000 - tp: 8.0000 - precision: 0.4706 - recall: 0.2000 - val_loss: 0.0066 - val_fn: 7.0000 - val_fp: 5.0000 - val_tn: 27.0000 - val_tp: 0.0000e+00 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 4/10
159/159 - 0s - loss: 0.0077 - fn: 33.0000 - fp: 15.0000 - tn: